# Text Embedding using Palm

In [4]:
import pandas as pd
import vertexai
from vertexai.preview.language_models import TextGenerationModel,TextEmbeddingModel

In [5]:
PROJECT_ID= "demogct2022"
Region = "us-central1"

In [6]:
vertexai.init(project=PROJECT_ID,location=Region)

In [7]:
model = TextEmbeddingModel.from_pretrained("textembedding-gecko@001")

In [8]:
def gen_embeddings( wikipedia_strings):
 index = 0
 allembeddings = []
 for i in range(5,len (wikipedia_strings)+1,5):
   print(i)
   embedarr = model.get_embeddings(wikipedia_strings[index:i] )
   allembeddings.append(embedarr)
   index = i
 return allembeddings


In [9]:
import pickle

In [17]:
with open("metadata_list_txt_uptd.jsonl", "rb") as f:
    metadata_df = pickle.load(f)

In [19]:
promptlist = [text["prompts"] for text in metadata_df]


In [21]:
len(promptlist)

114350

In [35]:
from IPython.display import clear_output

def gen_embeddings( promptlist):
 index = 0
 allembeddings = []
 errors = []
 for i in range(5,len (promptlist)+1,5):
   print(i)
   try:
    embedarr = model.get_embeddings(promptlist[index:i] )
    allembeddings.append(embedarr)
    index = i
    if index % 50 == 0:
        clear_output(wait=True)
    if index % 1000 == 0:
     with open("allembeddings.pkl", "wb") as f:
        pickle.dump(allembeddings,f)
        print("Saved a backup")
   except:
    errors.append(i)
    print(f"Error in index {i}")
    
 return allembeddings, errors

def flatten(allembeddings):
 flatlist = []
 for l in allembeddings:
    for e in l:
        flatlist.append(e.values)
 return flatlist

In [37]:
embs,errors = gen_embeddings(promptlist)

114305
114310
114315
114320
114325
114330
114335
114340
114345
114350


In [38]:
emblist = flatten(embs)

In [39]:
len(emblist)

114350

In [40]:
def gen_emb_csv(medf, csvfname):
 with open(csvfname, "w") as f:
    for i in range(medf.embeddings.size):
        f.write( str(i) + ',')
        f.write( ",".join(str(medf.embeddings[i])[1:-1].split(",")))
        f.write("\n")

In [41]:
medf = pd.DataFrame()
medf['embeddings'] = emblist
medf['promptlist'] = promptlist
medf.head()

,embeddings,promptlist
0,"[0.022194404155015945, 0.012218287214636803, 0...",**<https://s.mj.run/Wi_J4gq275A> guy fawkes ma...
1,"[-0.01536139938980341, -0.026366332545876503, ...",**an abstract print of 3 brown and white caval...
2,"[0.036830682307481766, 0.011287277564406395, 0...","**guy fawkes mask, Victo Ngai art nouveau, --a..."
3,"[-0.006090594921261072, -0.004152672830969095,...",**an abstract etched print of 3 brown and whit...
4,"[-0.042925961315631866, -0.03103814460337162, ...",**<https://s.mj.run/m4zXQ5v3ZCo> medieval fort...


In [42]:
gen_emb_csv(medf,"114k_PALM_Embedding.csv")

In [45]:
!gsutil cp 114k_PALM_Embedding.csv gs://vertex-gen-ai/mjembedding/text/palm/

Copying file://114k_PALM_Embedding.csv [Content-Type=text/csv]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

| [1 files][  1.8 GiB/  1.8 GiB]   54.7 MiB/s                                   
Operation completed over 1 objects/1.8 GiB.                                      
